In [11]:
import pandas as pd
import os
import os.path as osp
from IPython.display import display 
import numpy as np

In [12]:
def change_to_root_location() -> None:
    current_path = os.getcwd()
    if "pyproject.toml" in os.listdir(current_path) :
        print("Already in root location: ", current_path)
    else:
        os.chdir("..")
        change_to_root_location()

# Make sure that this notebook is located at the root of the project
change_to_root_location()

Already in root location:  /home/sebastiangarcia/Documents/u/8/bd-lab/semana-10/ingesta_datos


In [13]:
import typing as ty
DATA_FOLDER = "data/ml-100k"

genres_columns = [
    'unknown', 'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
    'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
    'Thriller', 'War', 'Western'
]

def read_dataset(data_folder: str, genres_columns: ty.List[str]) -> ty.Tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame]:
    movies_path = osp.join(data_folder, "u.item")
    user_path = osp.join(data_folder, "u.user")
    ratings_path = osp.join(data_folder, "u.data")


    movies_df = pd.read_csv(movies_path, sep='|', encoding='latin-1', header=None,
                           names=['movie_id', 'title', 'release_date', 'video_release_date', 'IMDb_URL', 'unknown',
                                  'Action', 'Adventure', 'Animation', 'Children', 'Comedy', 'Crime', 'Documentary',
                                  'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi',
                                  'Thriller', 'War', 'Western'])
    movies_df[genres_columns] = movies_df[genres_columns].astype(np.int8)
    # Read users.data file into a DataFrame
    users_df = pd.read_csv(user_path, sep='|', encoding='latin-1', header=None,
                          names=['user_id', 'age', 'gender', 'occupation', 'zip_code'])

    # Read ratings.data file into a DataFrame
    ratings_df = pd.read_csv(ratings_path, sep='\t', encoding='latin-1', header=None,
                            names=['user_id', 'movie_id', 'rating', 'timestamp'])

    # Display the first few rows of each DataFrame
    print('Movies DataFrame:')
    display(movies_df.head())
    print('\nUsers DataFrame:')
    display(users_df.head())
    print('\nRatings DataFrame:')
    display(ratings_df.head())
    return movies_df, users_df, ratings_df

def group_genres(movies_df: pd.DataFrame, genres_columns: ty.List[str]) -> pd.DataFrame:
    movies_df['genre'] = movies_df[genres_columns].apply(lambda x: ','.join(x[x==1].index), axis=1)
    movies_df['num_genres'] = movies_df[genres_columns].apply(lambda x: len((x[x==1].index)), axis=1)
    # Drop the individual genre columns
    movies_df = movies_df.drop(columns=genres_columns)
    return movies_df

movies_df, users_df, ratings_df = read_dataset(DATA_FOLDER, genres_columns)

Movies DataFrame:


,movie_id,title,release_date,video_release_date,IMDb_URL,unknown,Action,Adventure,Animation,Children,...,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,...,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0



Users DataFrame:


,user_id,age,gender,occupation,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213



Ratings DataFrame:


,user_id,movie_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [14]:
genres = pd.DataFrame(dict(genre_name=genres_columns)).reset_index(drop=False)
genres.rename(columns={'index': 'genre_id'}, inplace=True)
genres

genres_2_id = dict(zip(genres.genre_name, genres.genre_id))
genres_2_id

{'unknown': 0,
 'Action': 1,
 'Adventure': 2,
 'Animation': 3,
 'Children': 4,
 'Comedy': 5,
 'Crime': 6,
 'Documentary': 7,
 'Drama': 8,
 'Fantasy': 9,
 'Film-Noir': 10,
 'Horror': 11,
 'Musical': 12,
 'Mystery': 13,
 'Romance': 14,
 'Sci-Fi': 15,
 'Thriller': 16,
 'War': 17,
 'Western': 18}

In [15]:
movies_df = group_genres(movies_df, genres_columns)

In [16]:
movies_df

,movie_id,title,release_date,video_release_date,IMDb_URL,genre,num_genres
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,"Animation,Children,Comedy",3
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,"Action,Adventure,Thriller",3
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,Thriller,1
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,"Action,Comedy,Drama",3
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),"Crime,Drama,Thriller",3
...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,Drama,1
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,"Romance,Thriller",2
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),"Drama,Romance",2
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,Comedy,1


In [17]:
def create_movie_genre_table(movies_df: pd.DataFrame, genres_2_id: ty.Dict[str, int]) -> pd.DataFrame:
    movie_genres = dict(movie_id=[], genre_id=[])

    for _, row in movies_df.iterrows():
        genres = row['genre'].split(',')
        for genre in genres:
            movie_genres["movie_id"].append(row['movie_id'])
            movie_genres['genre_id'].append(genres_2_id[genre])

    movie_genres_df = pd.DataFrame(movie_genres)
    return movie_genres_df

In [18]:
movie_genres_df = create_movie_genre_table(movies_df, genres_2_id)
movie_genres_df

,movie_id,genre_id
0,1,3
1,1,4
2,1,5
3,2,1
4,2,2
...,...,...
2888,1679,16
2889,1680,8
2890,1680,14
2891,1681,5


In [19]:
movies_df.drop(columns=['genre'], inplace=True)
movies_df

,movie_id,title,release_date,video_release_date,IMDb_URL,num_genres
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,3
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,3
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,1
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,3
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),3
...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,1
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,2
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),2
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,1


In [20]:
movies_df.to_csv(osp.join(DATA_FOLDER, "processed_movies.csv"), index=False)

In [22]:
import os.path as osp

with open("ddl.sql", 'r') as f:
    ddl = f.read()

print(ddl.split(";")[-2])



CREATE TABLE ratings (
  rating_id INT PRIMARY KEY,
  user_id INT,
  movie_id INT,
  rating INT,
  timestamp INT,
  FOREIGN KEY (user_id) REFERENCES users(user_id),
  FOREIGN KEY (movie_id) REFERENCES movies(movie_id)
)


In [32]:
# get data as tuples
data = []

for record in movies_df.to_records(index=False):
    data.append(tuple(record))

data[0]

(1,
 'Toy Story (1995)',
 '01-Jan-1995',
 nan,
 'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)',
 3)

In [35]:
movies_data = [tuple(x) for x in movies_df.values]
movies_data[0]

(1,
 'Toy Story (1995)',
 '01-Jan-1995',
 nan,
 'http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)',
 3)

In [46]:
movies_df['release_date'].astype("datetime64[ns]").astype(str).fillna(lambda x: None)

0       1995-01-01
1       1995-01-01
2       1995-01-01
3       1995-01-01
4       1995-01-01
           ...    
1677    1998-02-06
1678    1998-02-06
1679    1998-01-01
1680    1994-01-01
1681    1996-03-08
Name: release_date, Length: 1682, dtype: object

In [47]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1682 entries, 0 to 1681
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   movie_id            1682 non-null   int64  
 1   title               1682 non-null   object 
 2   release_date        1681 non-null   object 
 3   video_release_date  0 non-null      float64
 4   IMDb_URL            1679 non-null   object 
 5   num_genres          1682 non-null   int64  
dtypes: float64(1), int64(2), object(3)
memory usage: 79.0+ KB


In [36]:
movies_df['release_date'] = movies_df['release_date'].fillna()

ValueError: Must specify a fill 'value' or 'method'.